# Simulated EEG Feature Extraction

This notebook demonstrates simulating and extracting features from EEG (Electroencephalography) data for real-time brain-music applications.

## Background
- **EEG** measures electrical activity via scalp electrodes
- **Temporal resolution**: ~1ms (excellent for real-time)
- **Spatial resolution**: Limited (volume conduction)
- **Coverage**: Primarily cortical surface

## Use Cases
- Real-time brain-music interfaces
- Detecting cognitive and emotional states
- Tracking temporal dynamics of creativity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("Libraries loaded successfully")

## 1. Simulate EEG Signals

Simulate multi-channel EEG during music listening with creative engagement.

In [ ]:
# Parameters
fs = 250  # Sampling frequency (Hz) - typical for EEG
duration = 60  # 1 minute recording
n_samples = fs * duration
time = np.arange(n_samples) / fs

# Electrode locations (simplified)
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
n_channels = len(channels)

print(f"Simulating EEG: {n_channels} channels, {duration}s, {fs}Hz")
print(f"Total samples: {n_samples}")

In [ ]:
def generate_eeg_band(freq_range, amplitude, time, fs):
    """Generate oscillatory activity in specific frequency band"""
    freq = np.random.uniform(freq_range[0], freq_range[1])
    phase = np.random.uniform(0, 2*np.pi)
    return amplitude * np.sin(2 * np.pi * freq * time + phase)

# EEG frequency bands
bands = {
    'delta': (1, 4),    # Deep sleep
    'theta': (4, 8),    # Meditation, creativity
    'alpha': (8, 13),   # Relaxed, closed eyes
    'beta': (13, 30),   # Active thinking, focus
    'gamma': (30, 50)   # High-level cognition
}

# Simulate EEG for each channel
eeg_data = np.zeros((n_channels, n_samples))

for ch in range(n_channels):
    # Each channel has different mix of frequency bands
    # Frontal channels: more theta/beta (executive function)
    # Parietal: more alpha (attention)
    # Occipital: more alpha (visual processing)
    
    if 'F' in channels[ch]:  # Frontal
        theta = generate_eeg_band(bands['theta'], 15, time, fs)
        beta = generate_eeg_band(bands['beta'], 8, time, fs)
        alpha = generate_eeg_band(bands['alpha'], 5, time, fs)
    elif 'C' in channels[ch]:  # Central
        theta = generate_eeg_band(bands['theta'], 10, time, fs)
        alpha = generate_eeg_band(bands['alpha'], 12, time, fs)
        beta = generate_eeg_band(bands['beta'], 8, time, fs)
    elif 'P' in channels[ch]:  # Parietal
        alpha = generate_eeg_band(bands['alpha'], 15, time, fs)
        theta = generate_eeg_band(bands['theta'], 8, time, fs)
        beta = generate_eeg_band(bands['beta'], 5, time, fs)
    else:  # Occipital
        alpha = generate_eeg_band(bands['alpha'], 18, time, fs)
        theta = generate_eeg_band(bands['theta'], 6, time, fs)
        beta = generate_eeg_band(bands['beta'], 4, time, fs)
    
    # Combine bands
    eeg_data[ch] = theta + alpha + beta
    
    # Add pink noise (1/f characteristic of EEG)
    pink_noise = np.cumsum(np.random.randn(n_samples)) / np.sqrt(fs)
    pink_noise = 5 * (pink_noise - np.mean(pink_noise)) / np.std(pink_noise)
    eeg_data[ch] += pink_noise
    
    # Add occasional artifacts (blinks, muscle)
    if 'Fp' in channels[ch]:  # Eye blinks in frontal channels
        blink_times = np.random.choice(n_samples, size=20, replace=False)
        for t in blink_times:
            blink = np.exp(-((np.arange(n_samples) - t)**2) / (0.1*fs)**2)
            eeg_data[ch] += 100 * blink

print("✓ EEG signals simulated")

In [ ]:
# Visualize raw EEG
fig, axes = plt.subplots(n_channels, 1, figsize=(14, 12))
plot_duration = 5  # Show first 5 seconds
plot_samples = int(plot_duration * fs)

for i, (ax, ch_name) in enumerate(zip(axes, channels)):
    ax.plot(time[:plot_samples], eeg_data[i, :plot_samples], linewidth=0.5)
    ax.set_ylabel(ch_name)
    ax.set_ylim(-50, 50)
    ax.grid(alpha=0.3)
    if i < n_channels - 1:
        ax.set_xticks([])

axes[-1].set_xlabel('Time (s)')
fig.suptitle('Simulated EEG Signals (First 5 seconds)', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

## 2. Power Spectral Density Analysis

Extract frequency band power - key features for EEG analysis.

In [ ]:
def compute_band_power(eeg_signal, fs, band_range):
    """Compute power in specific frequency band"""
    # Compute power spectral density
    freqs, psd = signal.welch(eeg_signal, fs, nperseg=2*fs)
    
    # Find indices for frequency band
    idx_band = np.logical_and(freqs >= band_range[0], freqs <= band_range[1])
    
    # Compute band power
    band_power = np.trapz(psd[idx_band], freqs[idx_band])
    return band_power

# Compute band powers for all channels
band_powers = {band: np.zeros(n_channels) for band in bands.keys()}

for ch in range(n_channels):
    for band_name, band_range in bands.items():
        band_powers[band_name][ch] = compute_band_power(eeg_data[ch], fs, band_range)

print("✓ Band powers computed for all channels")

In [ ]:
# Visualize band powers
fig, axes = plt.subplots(1, len(bands), figsize=(16, 4))

for ax, (band_name, powers) in zip(axes, band_powers.items()):
    ax.bar(channels, powers)
    ax.set_title(f'{band_name.capitalize()} ({bands[band_name][0]}-{bands[band_name][1]} Hz)')
    ax.set_ylabel('Power (µV²)')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Topographic visualization (simplified)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Show theta, alpha, beta as heatmaps
for ax, band_name in zip(axes, ['theta', 'alpha', 'beta']):
    # Simplified 2D layout
    topo_data = band_powers[band_name].reshape(2, 5)  # 2 rows, 5 cols
    im = ax.imshow(topo_data, cmap='RdYlBu_r', aspect='auto')
    ax.set_title(f'{band_name.capitalize()} Power')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## 3. Time-Frequency Analysis

Track how frequency content changes over time.

In [ ]:
# Compute spectrogram for one channel
channel_idx = 2  # F3 channel
freqs, times_spec, Sxx = signal.spectrogram(eeg_data[channel_idx], fs, 
                                            nperseg=2*fs, noverlap=fs)

# Plot spectrogram
plt.figure(figsize=(14, 6))
plt.pcolormesh(times_spec, freqs, 10 * np.log10(Sxx), 
               shading='gouraud', cmap='viridis')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.title(f'Spectrogram: {channels[channel_idx]}')
plt.ylim(0, 50)
plt.colorbar(label='Power (dB)')

# Mark frequency bands
for band_name, (low, high) in bands.items():
    plt.axhline(low, color='white', linestyle='--', alpha=0.3, linewidth=0.5)
    plt.axhline(high, color='white', linestyle='--', alpha=0.3, linewidth=0.5)

plt.tight_layout()
plt.show()

## 4. Extract Real-Time Features

Compute features suitable for real-time brain-music interfaces.

In [ ]:
def extract_realtime_features(eeg_window, fs, channels):
    """Extract features from EEG window for real-time use"""
    features = {}
    
    # 1. Band power ratios (indicators of cognitive state)
    theta_power = np.mean([compute_band_power(eeg_window[ch], fs, bands['theta']) 
                          for ch in range(len(channels))])
    alpha_power = np.mean([compute_band_power(eeg_window[ch], fs, bands['alpha']) 
                          for ch in range(len(channels))])
    beta_power = np.mean([compute_band_power(eeg_window[ch], fs, bands['beta']) 
                         for ch in range(len(channels))])
    
    features['theta_alpha_ratio'] = theta_power / (alpha_power + 1e-10)
    features['beta_alpha_ratio'] = beta_power / (alpha_power + 1e-10)
    
    # 2. Frontal alpha asymmetry (emotion/motivation)
    left_frontal = compute_band_power(eeg_window[2], fs, bands['alpha'])  # F3
    right_frontal = compute_band_power(eeg_window[3], fs, bands['alpha'])  # F4
    features['frontal_asymmetry'] = np.log(right_frontal) - np.log(left_frontal + 1e-10)
    
    # 3. Engagement index (beta/theta+alpha)
    features['engagement'] = beta_power / (theta_power + alpha_power + 1e-10)
    
    # 4. Overall activation (total power)
    features['activation'] = np.mean([np.var(eeg_window[ch]) for ch in range(len(channels))])
    
    return features

# Compute features over sliding windows
window_size = 2 * fs  # 2 second windows
hop_size = fs // 2  # 0.5 second hops
n_windows = (n_samples - window_size) // hop_size

feature_timeline = {
    'theta_alpha_ratio': [],
    'beta_alpha_ratio': [],
    'frontal_asymmetry': [],
    'engagement': [],
    'activation': []
}

for i in range(n_windows):
    start = i * hop_size
    end = start + window_size
    window = eeg_data[:, start:end]
    
    feats = extract_realtime_features(window, fs, channels)
    for key in feature_timeline.keys():
        feature_timeline[key].append(feats[key])

# Convert to arrays
for key in feature_timeline.keys():
    feature_timeline[key] = np.array(feature_timeline[key])

times_features = np.arange(n_windows) * (hop_size / fs) + (window_size / fs / 2)

print(f"✓ Extracted {len(feature_timeline)} features over {n_windows} windows")

In [ ]:
# Visualize real-time features
fig, axes = plt.subplots(5, 1, figsize=(14, 10))

feature_labels = {
    'theta_alpha_ratio': 'Theta/Alpha Ratio\n(Creativity/Relaxation)',
    'beta_alpha_ratio': 'Beta/Alpha Ratio\n(Attention/Relaxation)',
    'frontal_asymmetry': 'Frontal Alpha Asymmetry\n(Approach/Avoidance)',
    'engagement': 'Engagement Index\n(Beta/Theta+Alpha)',
    'activation': 'Overall Activation\n(Total Power)'
}

for ax, (key, label) in zip(axes, feature_labels.items()):
    ax.plot(times_features, feature_timeline[key], linewidth=2)
    ax.set_ylabel(label, fontsize=9)
    ax.grid(alpha=0.3)

axes[-1].set_xlabel('Time (s)')
fig.suptitle('Real-Time EEG Features Over Time', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

## Summary

### What We Did
1. Simulated realistic multi-channel EEG with frequency band structure
2. Computed power spectral density and band powers
3. Performed time-frequency analysis
4. Extracted real-time features for brain-music mapping:
   - Band power ratios (cognitive states)
   - Frontal asymmetry (emotion/motivation)
   - Engagement index (attention)
   - Overall activation

### Key Insights
- EEG provides **high temporal resolution** - ideal for real-time
- **Frequency bands** correspond to different cognitive states
- **Spatial patterns** reveal lateralized processes (emotion, attention)
- Features can be computed in **sliding windows** for continuous tracking

### Mapping to Music
These features can control musical parameters:
- **Theta/Alpha ratio** → Harmonic complexity (higher = more complex)
- **Engagement** → Tempo/energy (higher = faster/louder)
- **Frontal asymmetry** → Valence (positive = major, negative = minor)
- **Activation** → Overall intensity/density

### Next Steps
1. Map features to music latent spaces (see `06_latent_space_mapping.ipynb`)
2. Implement real-time processing pipeline (see `toy_interface/`)
3. Test with actual EEG hardware

### Advantages Over fMRI
- ✓ Real-time capability (<100ms latency possible)
- ✓ Portable and affordable
- ✓ Natural listening environment
- ✗ Limited spatial resolution
- ✗ Sensitive to artifacts